In [1]:
import os
import pandas as pd
import tgt
import glob
import tqdm

fs = glob.glob("it_vxc_textgrids17_acoustic17/common_voice_it_*.TextGrid")
tgrds = [tgt.read_textgrid(f) for f in fs]

In [2]:
from datasets import load_dataset
import datasets
ds = load_dataset("fsicoli/common_voice_22_0", "it")

all_paths = []
for split in ds.keys():
    ds[split] = ds[split].cast_column("audio", datasets.Audio(decode=False))
    paths = [i["path"] for i in ds[split].select(range(1))]
    all_paths.append("/".join(paths[0].split("/")[:-1]))

Using the latest cached version of the dataset since fsicoli/common_voice_22_0 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'it' at /Users/pohsuan/.cache/huggingface/datasets/fsicoli___common_voice_22_0/it/22.0.0/9d3b21d6c5398bd4e5196375ed1407a1b40d13bf44cb94a54675e9cd45ee4cca (last modified on Wed Nov 12 18:39:45 2025).


In [3]:
def find_f(target_id):
    for path in all_paths:
        f = os.path.join(path, f"common_voice_it_{target_id}.mp3")
        if os.path.exists(f):
            return f

In [ ]:
data = {
    "word": [],
    "id": [],
    "target_phone": [],
    "target_phone_start_time": [],
    "target_phone_end_time": [],
    "preceding_phone": [],
    "preceding_phone_start_time": [],
    "preceding_phone_end_time": [],
    "type": []
}
for f, tgrd in tqdm.tqdm(list(zip(fs, tgrds))):
    f_id = f.split("/")[-1].split(".")[0].split("_")[-1]
    for interval in tgrd.tiers[0].intervals:
        _type = None

        word = interval.text
        start_time = interval.start_time
        end_time = interval.end_time
        intervals = [interval for interval in tgrd.tiers[1].intervals if start_time <= interval.start_time < end_time]

        if len(intervals) < 2:
            continue
        if intervals[-1].text=="e":
            if intervals[-2].text in ("n", "m", "l", "r"):
                _type = "potential_troncamento"

        if intervals[-1].text in ("n", "m", "l", "r"):
            _type = "non_troncamento"

        if intervals[-1].text=="e" and intervals[-2].text not in ("n", "m", "l", "r"):
            _type = "non_troncamento"
        
        if _type is None:
            continue

        data["word"].append(word)
        data["id"].append(f_id)

        data["target_phone"].append(intervals[-1].text)
        data["target_phone_start_time"].append(intervals[-1].start_time)
        data["target_phone_end_time"].append(intervals[-1].end_time)
                
        data["preceding_phone"].append(intervals[-2].text)
        data["preceding_phone_start_time"].append(intervals[-2].start_time)
        data["preceding_phone_end_time"].append(intervals[-2].end_time)

        data["type"].append(_type)

100%|██████████| 68813/68813 [00:10<00:00, 6407.96it/s]


In [5]:
df = pd.DataFrame(data)
df["mp3_path"] = df["id"].apply(find_f)
df.dropna(subset=["mp3_path"], inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv("troncamento_data.csv", index=False)